# Exercise: Bayesian optimization with SilicoColorMixer

In this exercise you will use a Bayesian optimizer in a way similar to the way you used the SciPy optimizers in exercise 1 and 2. This exercise uses the `SilicoColorMixer`. Next exercise you will move to the LEGO robot, so make sure you do proper coding and experiment with parameters in this exercise to make the next one easier. 

You can [code your own Bayesian optimizer](https://machinelearningmastery.com/what-is-bayesian-optimization/), but unless you have something really specific in mind, you will often be better of using one of the many Bayesian optimizers readily available. Many of these in some way or another depend on [scikit-learn](https://scikit-learn.org/stable/index.html), a very popular Python machine learning library. 

In this exercise we will use a very advanced optimizer, [Dragonfly](https://dragonfly-opt.readthedocs.io/en/master/). If you are up for it you can read [the paper](https://arxiv.org/abs/1903.06694), but be aware that it is *really* difficult to grasp. Dragonfly is *not* user friendly, barely documented (which is sadly quite common for machine learning software), not updated for almost a year, and it is really computationally demanding, but it is able to reduce the number of function calls needed quite efficienctly. Since our function calls are rather slow when they involve physical experiments, this is a feature we really value. We could also have used the [Phoenics](https://pubs.acs.org/doi/full/10.1021/acscentsci.8b00307) optimizer.

Is this exercise you will learn to run the optimizer, load and save data, and apply advanced constraints. Dragonfly can do a lot more, but this will suffice.

Start by getting Dragonfly if you don't already have it.

In [ ]:
try:
    from dragonfly import minimize_function
    from argparse import Namespace
    import numpy as np
    from IPython import display
    import matplotlib.pyplot as plt
    from plot_pie_charts import make_piechart_plot
except:
    !pip3 install numpy --user --upgrade
    !pip3 install dragonfly-opt -v --user --upgrade
    from dragonfly import minimize_function
    from argparse import Namespace
    import numpy as np
    from IPython import display
    import matplotlib.pyplot as plt
    from plot_pie_charts import make_piechart_plot

Try running the simple example modified from the Dragonfly documentation below:

In [ ]:
def simple_function(x):
    val = x ** 4 - x**2 + 0.1 * x
    print(x, val) # To keep extra track on progress. Displays input and output
    return val

In [ ]:
simple_function_bounds = [[-5., 5.]]
max_func_calls = 20

In [ ]:
min_val, min_pt, history = minimize_function(simple_function, simple_function_bounds, max_func_calls)
print(min_val, min_pt)

Note that it is not deterministic and thus might arrive at different points if not fully converged.

Now, let us make it 2D. Our function should still only take one input, but it can be a list rather than a scalar.

In [ ]:
def simple_function2D(input_list):
    x, y = input_list[0], input_list[1]
    val = x ** 4 - x**2 + 0.1 * x + y**2
    print(input_list, val) # To keep extra track on progress. Displays input and output
    return val

Add a dimension to the bounds list.

In [ ]:
simple_function_bounds = [[-5., 5.], [-1., 1.]]
max_func_calls = 20

In [ ]:
min_val, min_pt, history = minimize_function(simple_function2D, simple_function_bounds, max_func_calls)
print(min_val, min_pt)

There you go. Conversion is slower with more parameters but this is really all it takes to start **using** the optimizer as a black box tool. We won't open the black box too much due to fear of what might come out if we do ;) but you will learn a few tricks to user it smarter.

To us, the magic number is 9 since we have 9 cuvettes to mix in when using the LEGO robot. Let us set some parameters to make the optimizer rebuild models for every 9 iterations and display a report line for every 9 as well. Let us also use 9 points as `init_capital`. That is the number of random points sampled before models are build.

Setting these parameters is really a matter or learning the Dragonfly convention and list of available parameters. Rather than having you spend time on this they will be spoon fed to you. But don't fall asleep. You will be put to the test shortly.

In [ ]:
options = Namespace(init_capital=9, build_new_model_every=9, report_results_every=9)

In [ ]:
min_val, min_pt, history = minimize_function(simple_function2D, simple_function_bounds, max_func_calls, options=options)
print(min_val, min_pt)

Another very useful feature is to be able to save progress and/or load in progress. `progress_load_from_and_save_to`, `progress_load_from` and `progress_save_to` will help you with this. You can use `progress_save_every` to save every *n* steps. Try is out.

In [ ]:
# The two top line finds your home directory
from pathlib import Path
home = str(Path.home())

options = Namespace(init_capital=9, build_new_model_every=9, report_results_every=9,
                    progress_load_from_and_save_to=home + '/47332/data/simple2D_savefile', progress_save_every=3)

In [ ]:
max_func_calls = 5

min_val, min_pt, history = minimize_function(simple_function2D, simple_function_bounds, max_func_calls, options=options)
print(min_val, min_pt)

In [ ]:
max_func_calls = 5

min_val, min_pt, history = minimize_function(simple_function2D, simple_function_bounds, max_func_calls, options=options)
print(min_val, min_pt)

# Moving to the color mixing

You have learned what you **need** to know about Dragonfly. Now it is your time to get to work. Reusing your code from the first *in silico* exercise, figure out how to mix the below target color using the Dragonfly optimizer.

You will use this code on the robot tomorrow, so be thorough and make sure to include the break after every 9 function calls. Plotting progress as you go will be very helpful in knowing when you feel satisfied with the optimization.

In [ ]:
target = (164., 176., 84.)

In [ ]:
# Cell for you to work in

In [ ]:
# Cell for you to work in

In [ ]:
# Cell for you to work in

In [ ]:
# Cell for you to work in. Add more as needed.

# Reusing old data

One thing is being able to reuse data generated with Dragonfly. Another thing is using data generated by other means. I have written a small helper function for you to help you with this. It is on purpose semi-manual to allow you to see what is going on. Let us go back to the 2D case to try it out.

In [ ]:
from make_savefile import make_dragonfly_save_file

Let us get some data. Normally you will not generate it but pull it out of a file.

In [ ]:
simple_function2D([2., 0.2])
simple_function2D([1., -0.2])
simple_function2D([-5., -1.])
simple_function2D([5., 1.])
simple_function2D([0., 0.])

Let us make that date into a list with the below format.

In [ ]:
data_list = [[2.0, 0.2, 12.239999999999998],
             [1.0, -0.2, 0.14],
             [-5.0, -1.0, 600.5],
             [5.0, 1.0, 601.5],
             [0.0, 0.0, 0.0]]

Dragonfly works in values relative to the bounds when saving data so we have to supply those as well. It also means that you should not use old savefiles when changing bounds, but you can always generate new savefiles from the raw results following the producere we examine here.

In [ ]:
constraints = [[-5., 5.], [-1., 1.]]

Lets make the file with data. 

In [ ]:
# The two top line finds your home directory
from pathlib import Path
home = str(Path.home())

filename = 'manual_data_simple_function2D'
file_path = home + '/47332/data/'
full_path = file_path + filename

make_dragonfly_save_file(data_list, full_path, constraints,)

Verify that you have written the file by looking at files in your data folder.

In [ ]:
!ls ~/47332/data/

Next, try using the file by loading it in.

In [ ]:
# The two top line finds your home directory
from pathlib import Path
home = str(Path.home())

options = Namespace(init_capital=9, build_new_model_every=9, report_results_every=9,
                    progress_load_from_and_save_to=home + '/47332/data/manual_data_simple_function2D', progress_save_every=3)

In [ ]:
max_func_calls = 5

min_val, min_pt, history = minimize_function(simple_function2D, constraints, max_func_calls, options=options)
print(min_val, min_pt)

When you do this trick (or load in data into Dragonfly in general), the initialization phase will be skipped. You should thus make sure that the data you load in is sufficiently diverse.

If you want practice and save time tomorrow, you can try this out with the SilicoColorMixer in the cells below.

In [ ]:
# Cell for you to work in.

In [ ]:
# Cell for you to work in.

In [ ]:
# Cell for you to work in.

In [ ]:
# Cell for you to work in. Add more as needed.

This concludes todays notebook.